## Point Cloud Classification for urban cover mapping
----------------------

The following workflow demonstrates point cloud classification using the Computational Geometry Algorithms Library This assumes basic knowledge of python and GIS usage. Compared to the other point cloud algorithms, this method is a little more sophisticated and could be applied to get a quick (i.e. minutes > hour generating training)  characterisation of an environment with relatively little training collection. 

The paper associated is found here:

https://hal.inria.fr/file/index/docid/759265/filename/ijcv_2012.pdf

The functions etc. of this workflow can be found here:

https://doc.cgal.org/latest/Classification/index.html

The algorithm is based on the computation of a set of pointwise geometric attributes (such as planarity, local elevation, etc.). Each type (heath, shrubs, trees) is defined as a linear combination of these attributes. Classification is achieved by minimizing an energy over the input point cloud by selecting, for each point, the classification type that gives the best score. Global regularization is performed by using a graph-cut algorithm (alpha expansion).

This dataset is a point cloud of the Cors Fochno SSSI building derived via SfM. The colours will look unusual as they represent the Red, Red-Edge & Near-Infra-Red bands repspectively, which are more informative for distinguising vegetation types. You will get used to it! 

Dependencies:

The code depends on the CGAL library, which can be installed most easily via anaconda or compiled from source.

Assumed knowledge:

Basic GIS, image processing, remote sensing, python and command line use

Remember to query function args open a new cell (the plus icon) and write the function with a question mark - i.e

function?

As the python CGAL API is a swig binding, docstring on functions may not be very informative. This workflow can also of course be easily executed directly using the C++ api.  

The lines of code have been written with readability in mind making each step as clear as possible, hence they are obviously not the briefest/most efficient way of doing things!

Data

The data is temporarily found in the QQ/KTP google drive.

The file required is Llandinam.zip - extract this somewhere appropriate and change dir to this folder.

https://drive.google.com/drive/folders/1vMV-ptTZ7WVrUi6VygYHbmFjXYE39QKl?usp=sharing

This contains:

- Llandinam.ply


**You will need a machine with plenty of cores and RAM to process this speedily.** 



The training labels are pre-made in the integer field "label". If you wish to create your own, the labels must be contiguous signed integers where -1 is nodata, 0,1,2,3 are the classes. This can be done in Cloud Compare (CC) by segmenting the point cloud and labeling with a scalar field. Better still, the whole operation can be run from the CGAL Polyhedron_3 demo GUI, if you follow the compilation instructions for this.

The CorsFochno.ply will look like this in CC.

<img src="Figures/Llandinam.png" style="height:500px">


### Module imports 
----------------------
The cgal lib can be found here with several ways to install. 

https://www.cgal.org/

The easiest is the official cgal repositories or anaconda. 
Install plyfile via pip in the same conda environment.  

https://www.cgal.org/download.html

https://anaconda.org/conda-forge/cgal

However this will not include the demos etc. See the repo README for some info on this.

In [2]:
from CGAL.CGAL_Kernel import Point_3
from CGAL.CGAL_Kernel import Vector_3
from CGAL.CGAL_Point_set_3 import Point_set_3
from CGAL.CGAL_Classification import *
from plyfile import PlyData
import sys
import os

Read in the data and return the no of points.

In [ ]:
incloud = 'Llandinam.ply'
points = Point_set_3(incloud)
print(points.size(), "points read")

### Class labels

This will enable us to extract the classes in the correct order for CGAL, though they could be predefined. This uses the ply header and pulls out the relevant text.

In [ ]:
pf = PlyData.read(incloud)
clList = pf.comments

# apologies for this ugly code!!!
if clList[0] == "Generated by the CGAL library":
    clss = clList[2:len(clList)]
    classes = [c.split()[2] for c in clss]
    classes.pop(0)
    del clss, clList

labels = Label_set()
for c in classes:
    labels.add(c)
    
print(classes)

If you wish to set different names here is how it is done. Uncomment this and set how you'd prefer.



In [4]:
#labels = Label_set()


#ground = labels.add("ground") # 0

#trees = labels.add("trees") # 1

#roof = labels.add("roof") # 2

#facade = labels.add("facade") # 3 

### Feature generation

The following lines generate point features based on the existing fields of the point cloud, eigen values and spatial filters.  

In [5]:
features = Feature_set()
generator = Point_set_feature_generator(points, 5)

features.begin_parallel_additions()
generator.generate_point_based_features(features)

if points.has_normal_map():
    generator.generate_normal_based_features(features, points.normal_map())


if points.has_int_map("red") and points.has_int_map("green") and points.has_int_map("blue"):
    generator.generate_color_based_features(features,
                                            points.int_map("red"),
                                            points.int_map("green"),
                                            points.int_map("blue"))

Add the example labels. 

In [6]:
classification = points.int_map("label")

### Training

Here we train the classifier and save the model for further use if required (and effective enough!). 

If you wish to label the training yourself - you will need either Cloud Compare or the CGAL Polyhedron_3 demo.

Cloud Compare will facillitate this by segment by a bit of a hacky workaround where you will have to segment bits of the cloud for each class, give them a constant scalar field (the class name) then merge them back together. This is not ideal but works! Here is a link to the tool basics.

https://www.cloudcompare.org/doc/wiki/index.php?title=Interactive_Segmentation_Tool

Better still, the CGAL_Polyhedron demo can be compliled from the main library but it may become a character building experience doing so. If successful you will have a QT-based GUI to label the data. Otherwise, there a few point labelling applications out there FOSS or proprietary but may not output the correct datatypes for CGAL!

Anyway.....

First we must create an path for the model to be saved - please insert your own. 


In [7]:
outmodel = "testrf.gz"
#outmodel = "/path/to/my/model.gz"

In [ ]:
training = points.add_int_map("training")
for idx in points.indices():
    training.set(idx, classification.get(idx))

print("Training random forest classifier...")
classifier = ETHZ_Random_forest_classifier(labels, features)
classifier.train(points.range(training), num_trees=50, max_depth=30)

print("Saving classifier's trained configuration...")
classifier.save_configuration(outmodel)

### Classification and output

Here we classify via a few different methods to see the merits/pitfalls of each.

1. The standard per-point classifier
2. Graphcut
3. Localised smoothing

Please replace the file paths with your own.

In [ ]:
std = "/path/to/std.ply"
gcut = "/path/to/gcut.ply"
lsmth = "/path/to/localsmooth.ply"

### Standard algorithm

Classifying per pixel, with no attempt at post process generalisation.

This may be noisier - but it is dependent on the task, classes etc. 

This will take the least amount of time.

In [ ]:
classify(points, labels, classifier, classification)
points.write(std)

### Localised smoothing

Kernelised smoothing....as above



In [ ]:
classify_with_local_smoothing(points, labels, classifier,
                                      generator.neighborhood().k_neighbor_query(6),
                                      classification)

points.write(lsmth)

### Graphcut

Using graphcut to approximate the optimal solution will result in a smoother classification - but may lose clarity. This will require more time to process



In [ ]:
classify_with_graphcut(points, labels, classifier,
                               generator.neighborhood().k_neighbor_query(6),
                               0.5,  # strength of graphcut
                               12,   # nb subdivisions (speed up)
                               classification)

points.write(gcut)

### Results 

You should end up with a map similar to this using the graphcut method, which for this example is the most effective. This could be improved upon with more training, but the results are largely correct (**the colours will be different in Cloud Compare - I have used the CGAL viewer as the colours are more intuitive and less unpleasant to look at!**).

- Yellow = Ground
- Green=Trees/Clutter
- Red = Roof
- Blue = Facade

<img src="Figures/LlandinamGcut.png" style="height:500px">




All of the functionality covered herein is in the script below. 

Where:

- i = the input cloud
- o = the output cloud
- m = the classification regularisation method
- f = fix the ply (cloud) file if you have used cloud compare to generate training (not used below)

In [ ]:
!cgalpointclassif.py -i 'Llandinam.ply' -o 'results.ply' -m 'graphcut'